In [1]:
# cpu only
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

## (1) Load model

In [2]:
from model_speedup_v1 import Mamba, ModelArgs
from transformers import AutoTokenizer

# One of:
#     'state-spaces/mamba-2.8b-slimpj'
#     'state-spaces/mamba-2.8b'
#     'state-spaces/mamba-1.4b'
#     'state-spaces/mamba-790m'
#     'state-spaces/mamba-370m'
#     'state-spaces/mamba-130m'
pretrained_model_name = 'state-spaces/mamba-370m'

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')
model, params = Mamba.from_pretrained(pretrained_model_name, tokenizer=tokenizer)

/home/shida/miniconda3/envs/S5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shida/miniconda3/envs/S5/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## (2) Generate Text

In [3]:
import jax
import jax.numpy as np

def jax_generate(model,
                 params, 
                 tokenizer,
                 prompt: str,
                 n_tokens_to_gen: int = 128,
                 sample: bool = True,
                 top_k: int = 40,
                 rng = jax.random.PRNGKey(7),
                 ):
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids # In pytorch format
    input_ids = np.array(input_ids.numpy()) # In jax format

    for token_n in range(n_tokens_to_gen):
        indices_to_input = input_ids
        next_token_logits = model.apply(params, indices_to_input)[:, -1]

        probs = jax.nn.softmax(next_token_logits, axis=-1)

        if top_k is not None:
            (values, indices) = jax.lax.top_k(probs, k=top_k)
            mask = probs < np.expand_dims(values[:, -1], axis=1)
            probs = np.where(mask, 0.0, probs)
            probs = probs / probs.sum(axis=1, keepdims=True)

        if sample:
            # TODO, might not be 100% correct. 
            rng, subrng = jax.random.split(rng)
            next_indices = jax.random.categorical(subrng, jax.nn.log_softmax(probs), 1, shape=probs.shape[:-1]+(1,))
        else:
            next_indices = np.argmax(probs, axis=-1, keepdims=True)

        input_ids = np.concatenate([input_ids, next_indices], axis=1)
    
    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

    return output_completions


In [4]:
sample=False

In [5]:
print(jax_generate(model, params, tokenizer, 'Mamba is the', sample=sample))

Mamba is the first game to be released on the Nintendo Switch. It is a side-scrolling platformer that is set in a futuristic world where the player must fight against the evil forces of the Mamba.

The game is set in a futuristic world where the player must fight against the evil forces of the Mamba. The game is set in a futuristic world where the player must fight against the evil forces of the Mamba.

The game is set in a futuristic world where the player must fight against the evil forces of the Mamba. The game is set in a futuristic world where the player must fight


In [6]:
print(jax_generate(model, params, tokenizer, 'John: Hi!\nSally:', sample=sample))

John: Hi!
Sally: Hi!
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John.
Sally: I'm Sally.
John: I'm John


In [7]:
print(jax_generate(model, params, tokenizer, 'The meaning of life is ', sample=sample))

KeyboardInterrupt: 

In [ ]:
print(jax_generate(model, params, tokenizer, 'def reverse_string(', sample=sample))